### This notebook serves to try and learn to use Binance

Establishing a connection to see the lastest bitcoin price

In [1]:
import requests

# Binance API endpoint for latest price
url = "https://api.binance.com/api/v3/ticker/price?symbol=BTCUSDT"

# Send request to Binance API
response = requests.get(url)

# Convert response to JSON
data = response.json()

# Print the latest price
print(f"Bitcoin Price: ${data['price']}")


Bitcoin Price: $93700.01000000


#### Exploring Binance Market Data
- Binance provides different types of data:

    - Latest Price (we already did this ✅)
    - Order Book (current buy/sell orders)
    - Trade History (list of past trades)
    - Candlestick (K-line) Data (historical price data over time) 🔥
    - 24h Ticker Data (price change in the last 24 hours)

##### Getting 24h Price Change Data

In [2]:
import requests

# Binance API endpoint for 24h ticker data
url = "https://api.binance.com/api/v3/ticker/24hr?symbol=BTCUSDT"

# Send request to Binance API
response = requests.get(url)
data = response.json()

# Extract useful information
print(f"Symbol: {data['symbol']}")
print(f"Price Change in 24h: {data['priceChange']} USDT")
print(f"Price Change Percentage: {data['priceChangePercent']}%")
print(f"24h High: {data['highPrice']} USDT")
print(f"24h Low: {data['lowPrice']} USDT")
print(f"24h Volume: {data['volume']} BTC")


Symbol: BTCUSDT
Price Change in 24h: -1731.82000000 USDT
Price Change Percentage: -1.816%
24h High: 96753.91000000 USDT
24h Low: 93388.09000000 USDT
24h Volume: 19839.67902000 BTC


#### What is Candlestick (K-line) Data?
Candlestick data is used in trading to represent price movements over a specific time period. Each "candle" contains:

- Open price (starting price at the beginning of the time period)
- High price (highest price during that period)
- Low price (lowest price during that period)
- Close price (final price at the end of that period)
- Volume (how much was traded)

In [16]:
import requests

# Binance API endpoint for candlestick data
url = "https://api.binance.com/api/v3/klines"

# Define request parameters
params = {
    "symbol": "ETHUSDT",  # Ethereum  to USDT
    "interval": "1d",  # Time interval (1 day)
    "limit": 100  # Get the last 100 data points
}

# Send request to Binance API
response = requests.get(url, params=params)
data = response.json()

first_candle = data[0]  # First candlestick data point

print(f"Timestamp (Open time): {first_candle[0]}")   # 0 - Open time (UNIX timestamp in milliseconds)
print(f"Open Price: {first_candle[1]} USDT")         # 1 - Open price
print(f"High Price: {first_candle[2]} USDT")         # 2 - High price
print(f"Low Price: {first_candle[3]} USDT")          # 3 - Low price
print(f"Close Price: {first_candle[4]} USDT")        # 4 - Close price
print(f"Volume: {first_candle[5]} ETH")              # 5 - Volume (ETH traded)
print(f"Close Time: {first_candle[6]}")              # 6 - Close time (UNIX timestamp in ms)
print(f"Quote Asset Volume: {first_candle[7]}")      # 7 - Quote asset volume (total USDT traded)
print(f"Number of Trades: {first_candle[8]}")        # 8 - Number of trades
print(f"Taker Buy Base Asset Volume: {first_candle[9]}")  # 9 - Volume of ETH bought by takers
print(f"Taker Buy Quote Asset Volume: {first_candle[10]}") # 10 - USDT spent by takers
print(f"Ignore: {first_candle[11]}")                 # 11 - Ignore (always 0, can be ignored)



2025-02-10 13:32:39.798 UTC DEBUG urllib3.connectionpool: Starting new HTTPS connection (1): api.binance.com:443
2025-02-10 13:32:40.250 UTC DEBUG urllib3.connectionpool: https://api.binance.com:443 "GET /api/v3/klines?symbol=ETHUSDT&interval=1d&limit=100 HTTP/11" 200 6287


Timestamp (Open time): 1730592000000
Open Price: 2494.23000000 USDT
High Price: 2496.39000000 USDT
Low Price: 2411.00000000 USDT
Close Price: 2457.73000000 USDT
Volume: 276009.11970000 ETH
Close Time: 1730678399999
Quote Asset Volume: 675516839.24615800
Number of Trades: 2569553
Taker Buy Base Asset Volume: 128755.13020000
Taker Buy Quote Asset Volume: 315138233.23078100
Ignore: 0


convert the UNIX timestamp to a human-readable format

In [17]:
from datetime import datetime

open_time = datetime.utcfromtimestamp(first_candle[0] / 1000)  # Convert ms to seconds
close_time = datetime.utcfromtimestamp(first_candle[6] / 1000)

print(f"Open Time: {open_time}")
print(f"Close Time: {close_time}")


Open Time: 2024-11-03 00:00:00
Close Time: 2024-11-03 23:59:59.999000


C:\Users\karim\AppData\Local\Temp\ipykernel_31380\2094123663.py:3: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  open_time = datetime.utcfromtimestamp(first_candle[0] / 1000)  # Convert ms to seconds
C:\Users\karim\AppData\Local\Temp\ipykernel_31380\2094123663.py:4: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  close_time = datetime.utcfromtimestamp(first_candle[6] / 1000)


#### How Binance Chooses the Start Date
When requesting 100 daily candles (interval="1d", limit=100), Binance:

- Uses the latest full daily candle as the last one.
- Counts 100 days backward to set the start date.
- So, if today is Feb 10, 2025, Binance returns data from Nov 3, 2024 to Feb 9, 2025 (100 days before today).

#### Controlling the Start Date Manually

In [18]:
import requests

url = "https://api.binance.com/api/v3/klines"

params = {
    "symbol": "ETHUSDT",
    "interval": "1d",
    "startTime": int(datetime(2024, 1, 1).timestamp() * 1000),  # Jan 1, 2024
    "endTime": int(datetime(2024, 2, 1).timestamp() * 1000),    # Feb 1, 2024
    "limit": 100
}

response = requests.get(url, params=params)
data = response.json()

print(f"First Candle Open Time: {datetime.utcfromtimestamp(data[0][0] / 1000)}")
print(f"Last Candle Open Time: {datetime.utcfromtimestamp(data[-1][0] / 1000)}")


2025-02-10 13:40:03.911 UTC DEBUG urllib3.connectionpool: Starting new HTTPS connection (1): api.binance.com:443
2025-02-10 13:40:04.596 UTC DEBUG urllib3.connectionpool: https://api.binance.com:443 "GET /api/v3/klines?symbol=ETHUSDT&interval=1d&startTime=1704063600000&endTime=1706742000000&limit=100 HTTP/11" 200 1938


First Candle Open Time: 2024-01-01 00:00:00
Last Candle Open Time: 2024-01-31 00:00:00


C:\Users\karim\AppData\Local\Temp\ipykernel_31380\443907220.py:16: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  print(f"First Candle Open Time: {datetime.utcfromtimestamp(data[0][0] / 1000)}")
C:\Users\karim\AppData\Local\Temp\ipykernel_31380\443907220.py:17: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  print(f"Last Candle Open Time: {datetime.utcfromtimestamp(data[-1][0] / 1000)}")


At specific hour

In [19]:
import requests
from datetime import datetime

url = "https://api.binance.com/api/v3/klines"

params = {
    "symbol": "ETHUSDT",
    "interval": "1h",
    "startTime": int(datetime(2025, 2, 9, 18, 0).timestamp() * 1000),  # Feb 9, 18:00 UTC
    "limit": 5
}

response = requests.get(url, params=params)
data = response.json()

for candle in data:
    print(f"Open Time: {datetime.utcfromtimestamp(candle[0] / 1000)}")


2025-02-10 13:47:47.936 UTC DEBUG urllib3.connectionpool: Starting new HTTPS connection (1): api.binance.com:443
2025-02-10 13:47:48.658 UTC DEBUG urllib3.connectionpool: https://api.binance.com:443 "GET /api/v3/klines?symbol=ETHUSDT&interval=1h&startTime=1739120400000&limit=5 HTTP/11" 200 336


Open Time: 2025-02-09 17:00:00
Open Time: 2025-02-09 18:00:00
Open Time: 2025-02-09 19:00:00
Open Time: 2025-02-09 20:00:00
Open Time: 2025-02-09 21:00:00


C:\Users\karim\AppData\Local\Temp\ipykernel_31380\1564835347.py:17: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  print(f"Open Time: {datetime.utcfromtimestamp(candle[0] / 1000)}")


### Finally :Fetch Binance Candlestick Data & Save to CSV

In [21]:
import requests
import pandas as pd
from datetime import datetime

# Binance API endpoint
url = "https://api.binance.com/api/v3/klines"

# Define request parameters
params = {
    "symbol": "ETHUSDT",  # Ethereum to USDT
    "interval": "1d",  # 1-day interval
    "limit": 365  # Get the last 100 candles
}

# Send request to Binance API
response = requests.get(url, params=params)
data = response.json()

# Convert to DataFrame
columns = ["Open_Time", "Open", "High", "Low", "Close", "Volume", 
           "Close_Time", "Quote_Asset_Volume", "Number_of_Trades", 
           "Taker_Buy_Base_Volume", "Taker_Buy_Quote_Volume", "Ignore"]

df = pd.DataFrame(data, columns=columns)

# Convert timestamps to human-readable format
df["Open_Time"] = pd.to_datetime(df["Open_Time"], unit="ms")
df["Close_Time"] = pd.to_datetime(df["Close_Time"], unit="ms")

# Save to CSV
csv_filename = "eth_usdt_candles.csv"
df.to_csv(csv_filename, index=False)

print(f"Data saved to {csv_filename}")


2025-02-10 13:54:58.442 UTC DEBUG urllib3.connectionpool: Starting new HTTPS connection (1): api.binance.com:443
2025-02-10 13:54:58.864 UTC DEBUG urllib3.connectionpool: https://api.binance.com:443 "GET /api/v3/klines?symbol=ETHUSDT&interval=1d&limit=365 HTTP/11" 200 22866


Data saved to eth_usdt_candles.csv


Let's make a function that takes the coin, interval and limit as arguments and outputs a dataframe for candles information

In [ ]:
import requests
import pandas as pd
from datetime import datetime


def get_lkines(symbol= ETHUSDT, interval= "1d", limit = 365):
    url = "https://api.binance.com/api/v3/klines"
    params = {
        "symbol": symbol,
        "interval": interval,
        "limit": limit
    }

    response = requests.get(url, params=params)
    data = response.json()
    # Convert to DataFrame
    columns = ["Open_Time", "Open", "High", "Low", "Close", "Volume", 
           "Close_Time", "Quote_Asset_Volume", "Number_of_Trades", 
           "Taker_Buy_Base_Volume", "Taker_Buy_Quote_Volume", "Ignore"]
    df = pd.DataFrame(data, columns=columns)
    # Convert timestamps to human-readable format
    df["Open_Time"] = pd.to_datetime(df["Open_Time"], unit="ms")
    df["Close_Time"] = pd.to_datetime(df["Close_Time"], unit="ms")
    return data


In [ ]:
# Coins map_ humanely readable extracted from notebook
symbol_map = {
    "bitcoin": "BTCUSDT",
    "ethereum": "ETHUSDT",
    "binance-coin": "BNBUSDT",
    "ripple": "XRPUSDT",
    "cardano": "ADAUSDT",
    "solana": "SOLUSDT",
    "polkadot": "DOTUSDT",
    "dogecoin": "DOGEUSDT",
    "shiba-inu": "SHIBUSDT",
    "litecoin": "LTCUSDT",
    "chainlink": "LINKUSDT",
    "polygon": "MATICUSDT",
    "avalanche": "AVAXUSDT",
    "uniswap": "UNIUSDT",
    "cosmos": "ATOMUSDT",
    "stellar": "XLMUSDT",
    "vechain": "VETUSDT",
    "filecoin": "FILUSDT",
    "algorand": "ALGOUSDT",
    "monero": "XMRUSDT",
    "bitcoin-cash": "BCHUSDT",
    "eos": "EOSUSDT",
    "tezos": "XTZUSDT",
    "aave": "AAVEUSDT",
    "compound": "COMPUSDT",
    "maker": "MKRUSDT",
    "theta": "THETAUSDT",
    "axie-infinity": "AXSUSDT",
    "decentraland": "MANAUSDT",
    "the-sandbox": "SANDUSDT",
    "gala": "GALAUSDT",
    "enjin-coin": "ENJUSDT",
    "curve-dao-token": "CRVUSDT",
    "yearn-finance": "YFIUSDT",
    "synthetix": "SNXUSDT",
    "uma": "UMAUSDT",
    "balancer": "BALUSDT",
    "ren": "RENUSDT",
    "0x": "ZRXUSDT",
    "loopring": "LRCUSDT",
    "harmony": "ONEUSDT",
    "kava": "KAVAUSDT",
    "zilliqa": "ZILUSDT",
    "icon": "ICXUSDT",
    "ontology": "ONTUSDT",
    "qtum": "QTUMUSDT",
    "iost": "IOSTUSDT",
    "waves": "WAVESUSDT",
    "omisego": "OMGUSDT",
    "nano": "NANOUSDT",
    "ravencoin": "RVNUSDT",
    "helium": "HNTUSDT",
    "chiliz": "CHZUSDT",
    "ankr": "ANKRUSDT",
    "fetch-ai": "FETUSDT",
    "numeraire": "NMRUSDT",
    "reserve-rights": "RSRUSDT",
    "band-protocol": "BANDUSDT",
    "ocean-protocol": "OCEANUSDT",
    "arweave": "ARUSDT",
    "celo": "CELOUSDT",
    "uma": "UMAUSDT",
    "mirror-protocol": "MIRUSDT",
    "injective-protocol": "INJUSDT",
    "alpha-finance": "ALPHAUSDT",
    "serum": "SRMUSDT",
    "raydium": "RAYUSDT",
    "orca": "ORCAUSDT",
    "audius": "AUDIOUSDT",
    "tellor": "TRBUSDT",
    "uma": "UMAUSDT",
    "perpetual-protocol": "PERPUSDT",
    "dydx": "DYDXUSDT",
    "illuvium": "ILVUSDT",
    "radix": "XRDUSDT",
    "mina": "MINAUSDT",
    "secret": "SCRTUSDT",
    "juno": "JUNOUSDT",
    "osmosis": "OSMOUSDT",
    "evmos": "EVMOSUSDT",
    "stargaze": "STARSUSDT",
    "persistence": "XPRTUSDT",
    "regen": "REGENUSDT",
    "akash": "AKTUSDT",
    "sentinel": "DVPNUSDT",
    "irisnet": "IRISUSDT",
    "kava": "KAVAUSDT",
    "terra": "LUNAUSDT",  # Note: Terra (LUNA) has been relaunched after the crash
    "anchor-protocol": "ANCUSDT",
    "mirror-protocol": "MIRUSDT",
    "thorchain": "RUNEUSDT",
    "kadena": "KDAUSDT",
    "hedera-hashgraph": "HBARUSDT",
    "elrond": "EGLDUSDT",
    "near-protocol": "NEARUSDT",
    "flow": "FLOWUSDT",
    "klaytn": "KLAYUSDT",
    "fantom": "FTMUSDT",
    "pancakeswap": "CAKEUSDT",
    "sushi": "SUSHIUSDT",
    "1inch": "1INCHUSDT",
    "bancor": "BNTUSDT",
    "kyber-network": "KNCUSDT",
    "uma": "UMAUSDT",
    "uma": "UMAUSDT",
    "uma": "UMAUSDT",
}